In [1]:
import pandas as pd
import numpy as np

In [2]:
import spacy

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

In [5]:
df.head()

,url,sent_num,sentence,text,label
0,http://www.newindianexpress.com/states/andhra-...,1-1,Anantapur Lawyers to Revive Samaikya Agitation...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
1,http://www.newindianexpress.com/states/andhra-...,1-2,A meeting of lawyers was held in the town on S...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
2,http://www.newindianexpress.com/states/andhra-...,1-3,"Speaking to reporters later, advocates’ leader...",Anantapur Lawyers to Revive Samaikya Agitation...,0.0
3,http://www.newindianexpress.com/states/andhra-...,1-4,“Bifurcation is being done without taking into...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
4,http://www.newindianexpress.com/states/andhra-...,1-5,The Bill did not contain any proposals for dev...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0


In [6]:
mask = np.logical_not(np.isnan(np.array(df['label'])))

In [7]:
cleandf = df[mask]

In [8]:
len(cleandf)

8337

## Nlp pipeline

In [9]:
nlp = spacy.load('en')

In [10]:
# http://www.insightsbot.com/blog/R8fu5/bag-of-words-algorithm-in-python-introduction
def extract_words(sentence):
    ignore_words = ['a']
    #words = re.sub("[^\w]", " ",  sentence).split() #nltk.word_tokenize(sentence)
    doc = nlp(sentence)
    words = [token.lemma_ for token in doc]
    words_cleaned = [w.lower().strip() for w in words if w not in ignore_words]
    return words_cleaned  

def tokenize_sentences(sentences):
    words = []
    for sentence in sentences:
        w = extract_words(sentence)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

def bagofwords(sentence, words):
    sentence_words = extract_words(sentence)
    # frequency word count
    bag = np.zeros(len(words))
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag)

In [11]:
vocab = tokenize_sentences(cleandf['sentence'])

KeyboardInterrupt: 

In [ ]:
import pickle
with open('Data/vocabulary.pickle', 'wb') as handle:
    pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

vocab = pd.read_pickle("./vocabulary.pickle")

In [16]:
cleandf = cleandf.reset_index(drop=True)

In [33]:
vectorizer = TfidfVectorizer(min_df=0.002, max_df=0.95, stop_words='english')
tfidf_vectors = vectorizer.fit_transform(cleandf['sentence'])
tfidf_vectors

<8337x1320 sparse matrix of type '<class 'numpy.float64'>'
	with 68153 stored elements in Compressed Sparse Row format>

In [34]:
len(vectorizer.get_feature_names())

1320

In [27]:
X = tfidf_vectors
y = np.array(cleandf['label'])

In [55]:
# Feature Selection
feature_selector = SelectPercentile(chi2, percentile=70)
X_new = feature_selector.fit_transform(tfidf_vectors, y)


In [45]:
X.shape

(8337, 1032)

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectPercentile, chi2

In [28]:

clf = DecisionTreeClassifier(criterion='gini', max_depth=20, min_samples_split=100)
clf = clf.fit(X, y)

In [29]:

#write your code here
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, clf.predict(X))
print('We classified ', accuracy, ' of the data correctly')

We classified  0.8766942545280076  of the data correctly


In [54]:
X_newnew = vectorizer.inverse_transform(X_new)
X_newnew

[array(['01', '09', '100', '14th', '2014'], dtype='<U18'),
 array(['01st', '2014'], dtype='<U18'),
 array([], dtype='<U18'),
 array([], dtype='<U18'),
 array([], dtype='<U18'),
 array(['09', '100', '2014'], dtype='<U18'),
 array(['2010', '25'], dtype='<U18'),
 array(['09', '100', '14th', '22nd'], dtype='<U18'),
 array(['01st', '100', '11', '14th'], dtype='<U18'),
 array(['14th'], dtype='<U18'),
 array(['09'], dtype='<U18'),
 array(['09'], dtype='<U18'),
 array([], dtype='<U18'),
 array(['01', '07th', '11th', '12', '19th', '26th', '29th'], dtype='<U18'),
 array(['14th'], dtype='<U18'),
 array(['2008'], dtype='<U18'),
 array(['00', '08th', '100', '14th'], dtype='<U18'),
 array([], dtype='<U18'),
 array(['08th', '2015'], dtype='<U18'),
 array([], dtype='<U18'),
 array(['08th'], dtype='<U18'),
 array([], dtype='<U18'),
 array([], dtype='<U18'),
 array([], dtype='<U18'),
 array(['14th'], dtype='<U18'),
 array([], dtype='<U18'),
 array(['100'], dtype='<U18'),
 array([], dtype='<U18'),
 array

In [47]:

clf = DecisionTreeClassifier(criterion='gini', max_depth=20, min_samples_split=100)
clf = clf.fit(X_new, y)


#write your code here
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, clf.predict(X_new))
print('We classified ', accuracy, ' of the data correctly')

We classified  0.8705769461436967  of the data correctly


In [59]:
[*range(3,13,2)]

[3, 5, 7, 9, 11]

In [22]:
vectorizer = TfidfVectorizer()
tfidf_vectors = vectorizer.fit_transform(cleandf['sentence'])
tfidf_vectors

<8337x16604 sparse matrix of type '<class 'numpy.float64'>'
	with 175989 stored elements in Compressed Sparse Row format>

In [23]:
len(vectorizer.get_feature_names())

16604

In [24]:

clf = DecisionTreeClassifier(criterion='gini', max_depth=20, min_samples_split=100)
clf = clf.fit(X, y)


#write your code here
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, clf.predict(X))
print('We classified ', accuracy, ' of the data correctly')

We classified  0.8782535684298909  of the data correctly


In [103]:
type(tfidf_vectors)

scipy.sparse.csr.csr_matrix

In [17]:
def extract_entities(sentence):
    doc = nlp(row['sentence'])
    entities = [token.label_ for token in doc.ents]
    return entities  

def bagofentities(sentence, ner_tagset):
    entities = extract_entities(sentence)
    # frequency word count
    bag = np.zeros(len(ner_tagset))
    for ent in entities:
        for i, entity in enumerate(ner_tagset):
            if ent==entity:
                bag[i] += 1
    return np.array(bag)

In [20]:
feature_list = pd.read_pickle("./feature_list.pickle")

## Classifier Comparison

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectPercentile, chi2

In [24]:
# All classifiers
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [29]:
X = feature_list
y = np.array(cleandf['label'])

In [ ]:
# Feature Selection
feature_selector = SelectPercentile(chi2, percentile=70)
feature_selector.fit(X, y)


In [ ]:
list_of_new_features = []
for i, feature_name in enumerate(list(X)):
    if feature_selector.get_support()[i]:
        list_of_new_features.append(feature_name)

clf = SVC()#kernel='linear', C=1)
# scores = cross_val_score(clf, X, y, cv=5)

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [42]:
grid_clf = GridSearchCV(clf, parameters, cv=5, n_jobs=5)

In [ ]:
grid_clf.fit(X, y)

In [21]:
result_file = open("scikit_learn_results.txt","w") 


In [99]:
for name, classifier in zip(names[:2], classifiers[:2]):
    y_true, y_pred = y, cross_val_predict(classifier, X, y)
    result_file.write('\n---', name, '---')
    result_file.write('Precision:', precision_score(y_true, y_pred, average='weighted'))
    result_file.write('Recall:', recall_score(y_true, y_pred, average='weighted'))
    result_file.write('F1-score:', f1_score(y_true, y_pred, average='weighted'))
    result_file.write('Accuracy:', accuracy_score(y_true, y_pred))

/scratch/sekiz/DM/.env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


KeyboardInterrupt: 

In [22]:
for name, classifier in zip(names[:2], classifiers[:2]):
#     y_true, y_pred = y, cross_val_predict(classifier, X, y)
    result_file.write('\n--- ' + name + ' ---')
    result_file.write('Precision:')
    result_file.write('Recall:')
    result_file.write('F1-score:')
    result_file.write('Accuracy:')

In [23]:
result_file.close()